In [1]:
    %pip install opencv-python mediapipe ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import cv2

In [ ]:
import numpy as np

In [ ]:
from ultralytics import YOLO

In [ ]:
import mediapipe as mp

In [18]:
# final model for face/head and eye tracking


# Load YOLO model
model = YOLO("yolov8n.pt")  # Use a face-trained model if you have one

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Cannot open webcam.")
    exit()

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

def get_head_pose(image, landmarks):
    image_points = np.array([
        landmarks[1],    # Nose tip
        landmarks[152],  # Chin
        landmarks[33],   # Right eye outer corner
        landmarks[263],  # Left eye outer corner
        landmarks[78],   # Right mouth corner
        landmarks[308]   # Left mouth corner
    ], dtype="double")

    model_points = np.array([
        (0.0, 0.0, 0.0),             # Nose tip
        (0.0, -330.0, -65.0),        # Chin
        (225.0, 170.0, -135.0),      # Right eye outer corner
        (-225.0, 170.0, -135.0),     # Left eye outer corner
        (150.0, -150.0, -125.0),     # Right mouth corner
        (-150.0, -150.0, -125.0)     # Left mouth corner
    ])

    focal_length = image.shape[1]
    center = (image.shape[1] / 2, image.shape[0] / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))
    success, rvec, _, _ = cv2.solvePnPRansac(
        model_points, image_points, camera_matrix, dist_coeffs)

    rmat, _ = cv2.Rodrigues(rvec)
    proj_matrix = np.hstack((rmat, np.zeros((3, 1))))
    _, _, _, _, _, _, angles = cv2.decomposeProjectionMatrix(proj_matrix)

    return angles  # pitch, yaw, roll

# ✅ Updated eye direction using iris tracking
def get_eye_direction(landmarks, iw):
    try:
        right_iris_center = landmarks[468]  # Iris center
        right_eye_inner = landmarks[133]
        right_eye_outer = landmarks[33]

        eye_width = right_eye_outer[0] - right_eye_inner[0]
        if eye_width == 0:
            return "Undetected"

        iris_relative_pos = (right_iris_center[0] - right_eye_inner[0]) / eye_width

        if iris_relative_pos < 0.40:
            return "Looking Left"
        elif iris_relative_pos > 0.60:
            return "Looking Right"
        else:
            return "Looking Center"
    except:
        return "Undetected"

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame)

    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        face = frame[y1:y2, x1:x2]

        if face.shape[0] == 0 or face.shape[1] == 0:
            continue

        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        result_mesh = face_mesh.process(face_rgb)

        if result_mesh.multi_face_landmarks:
            for landmarks in result_mesh.multi_face_landmarks:
                ih, iw, _ = face.shape
                coords = [(p.x * iw, p.y * ih) for p in landmarks.landmark]

                try:
                    angles = get_head_pose(face, coords)
                    pitch, yaw, roll = [a[0] for a in angles]

                    if abs(yaw) > 10:  # ✅ Increased yaw threshold for more reliable detection
                        cheat = "⚠️ Head turned!"
                        print(f"Yaw: {yaw:.2f} - Cheat: {cheat}")
                    elif abs(pitch) > 7:
                        cheat = "⚠️ Looking up/down!"
                        print(f"Pitch: {pitch:.2f} - Cheat: {cheat}")
                    else:
                        cheat = ""
                except:
                    cheat = "Head Pose Error"
                    print("Head Pose Error")

                eye_dir = get_eye_direction(coords, iw)
                if eye_dir != "Looking Center":
                    cheat = f"⚠️ {eye_dir}"
                    print(f"Eye Direction: {eye_dir} - Cheat: {cheat}")
                else:
                    cheat = ""
                    print("Eye Direction: Looking Center")

                # Draw box and alert
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, eye_dir, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                if cheat:
                    cv2.putText(frame, cheat, (x1, y2 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("YOLO + MediaPipe Cheat Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.

0: 480x640 3 persons, 1 kite, 1 clock, 233.0ms
Speed: 5.7ms preprocess, 233.0ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)
Yaw: 27.08 - Cheat: ⚠️ Head turned!
Eye Direction: Looking Center
Pitch: 13.34 - Cheat: ⚠️ Looking up/down!
Eye Direction: Looking Center
Yaw: 23.62 - Cheat: ⚠️ Head turned!
Eye Direction: Looking Left - Cheat: ⚠️ Looking Left

0: 480x640 2 persons, 1 clock, 187.5ms
Speed: 3.8ms preprocess, 187.5ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)
Yaw: 42.56 - Cheat: ⚠️ Head turned!
Eye Direction: Looking Left - Cheat: ⚠️ Looking Left

0: 480x640 2 persons, 2 clocks, 211.2ms
Speed: 2.7ms preprocess, 211.2ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)
Yaw: 23.15 - Cheat: ⚠️ Head turned!
Eye Direction: Looking Center
Yaw: 10.44 - Cheat: ⚠️ Head turned!
Eye Direction: Looking Center

0: 480x640 2 persons, 1 clock, 191.1ms
Speed: 2.9ms preprocess, 191.1ms inference, 2.9ms postprocess per imag

In [11]:
# # model for both eye and face but need some changes

# import cv2
# import numpy as np
# from ultralytics import YOLO
# import mediapipe as mp

# # Load YOLO model
# model = YOLO("yolov8n.pt")  # Use a face-trained model if you have one

# # Initialize webcam
# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     print("Error: Cannot open webcam.")
#     exit()

# # Initialize MediaPipe Face Mesh
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
#                                   max_num_faces=1,
#                                   refine_landmarks=True,
#                                   min_detection_confidence=0.5,
#                                   min_tracking_confidence=0.5)

# def get_head_pose(image, landmarks):
#     image_points = np.array([
#         landmarks[1],    # Nose tip
#         landmarks[152],  # Chin
#         landmarks[33],   # Right eye outer corner
#         landmarks[263],  # Left eye outer corner
#         landmarks[78],   # Right mouth corner
#         landmarks[308]   # Left mouth corner
#     ], dtype="double")

#     model_points = np.array([
#         (0.0, 0.0, 0.0),             # Nose tip
#         (0.0, -330.0, -65.0),        # Chin
#         (225.0, 170.0, -135.0),      # Right eye outer corner
#         (-225.0, 170.0, -135.0),     # Left eye outer corner
#         (150.0, -150.0, -125.0),     # Right mouth corner
#         (-150.0, -150.0, -125.0)     # Left mouth corner
#     ])

#     focal_length = image.shape[1]
#     center = (image.shape[1] / 2, image.shape[0] / 2)
#     camera_matrix = np.array([
#         [focal_length, 0, center[0]],
#         [0, focal_length, center[1]],
#         [0, 0, 1]
#     ], dtype="double")

#     dist_coeffs = np.zeros((4, 1))
#     success, rvec, _, _ = cv2.solvePnPRansac(
#         model_points, image_points, camera_matrix, dist_coeffs)

#     rmat, _ = cv2.Rodrigues(rvec)
#     proj_matrix = np.hstack((rmat, np.zeros((3, 1))))
#     _, _, _, _, _, _, angles = cv2.decomposeProjectionMatrix(proj_matrix)

#     return angles  # pitch, yaw, roll

# # ✅ Updated eye direction using iris tracking
# def get_eye_direction(landmarks, iw):
#     try:
#         right_iris_center = landmarks[468]  # Iris center
#         right_eye_inner = landmarks[133]
#         right_eye_outer = landmarks[33]

#         eye_width = right_eye_outer[0] - right_eye_inner[0]
#         if eye_width == 0:
#             return "Undetected"

#         iris_relative_pos = (right_iris_center[0] - right_eye_inner[0]) / eye_width

#         if iris_relative_pos < 0.35:
#             return "Looking Left"
#         elif iris_relative_pos > 0.65:
#             return "Looking Right"
#         else:
#             return "Looking Center"
#     except:
#         return "Undetected"

# print("Press 'q' to quit.")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = model(frame)

#     for box in results[0].boxes.xyxy:
#         x1, y1, x2, y2 = map(int, box)
#         face = frame[y1:y2, x1:x2]

#         if face.shape[0] == 0 or face.shape[1] == 0:
#             continue

#         face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
#         result_mesh = face_mesh.process(face_rgb)

#         if result_mesh.multi_face_landmarks:
#             for landmarks in result_mesh.multi_face_landmarks:
#                 ih, iw, _ = face.shape
#                 coords = [(p.x * iw, p.y * ih) for p in landmarks.landmark]

#                 try:
#                     angles = get_head_pose(face, coords)
#                     pitch, yaw, roll = [a[0] for a in angles]
#                     if abs(yaw) > 5:
#                         cheat = "⚠️ Head turned!"
#                         print(f"Yaw: {yaw:.2f} - Cheat: {cheat}")
#                     elif abs(pitch) > 7:
#                         cheat = "⚠️ Looking up/down!"
#                         print(f"Pitch: {pitch:.2f} - Cheat: {cheat}")
#                     else:
#                         cheat = ""
#                 except:
#                     cheat = "Head Pose Error"
#                     print("Head Pose Error")

#                 eye_dir = get_eye_direction(coords, iw)
#                 if eye_dir != "Looking Center":
#                     cheat = f"⚠️ {eye_dir}"
#                     print(f"Eye Direction: {eye_dir} - Cheat: {cheat}")
#                 else:
#                     cheat = ""
#                     print("Eye Direction: Looking Center")

#                 # Draw box and alert
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 cv2.putText(frame, eye_dir, (x1, y1 - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
#                 if cheat:
#                     cv2.putText(frame, cheat, (x1, y2 + 20),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

#     cv2.imshow("YOLO + MediaPipe Cheat Detection", frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [12]:
#just for direction but jub me left dekh rha hun to bta sahi rha h looking right the issue is k wo bta k phir center hi bta rha h k 
#phle looking right phir jldi me looking left 

import cv2
import numpy as np
from ultralytics import YOLO
import mediapipe as mp

# Load YOLO model
model = YOLO("yolov8n.pt")  # Use a face-trained model if you have one

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Cannot open webcam.")
    exit()

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

def get_head_pose(image, landmarks):
    image_points = np.array([
        landmarks[1],    # Nose tip
        landmarks[152],  # Chin
        landmarks[33],   # Right eye outer corner
        landmarks[263],  # Left eye outer corner
        landmarks[78],   # Right mouth corner
        landmarks[308]   # Left mouth corner
    ], dtype="double")

    model_points = np.array([
        (0.0, 0.0, 0.0),             # Nose tip
        (0.0, -330.0, -65.0),        # Chin
        (225.0, 170.0, -135.0),      # Right eye outer corner
        (-225.0, 170.0, -135.0),     # Left eye outer corner
        (150.0, -150.0, -125.0),     # Right mouth corner
        (-150.0, -150.0, -125.0)     # Left mouth corner
    ])

    focal_length = image.shape[1]
    center = (image.shape[1] / 2, image.shape[0] / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))
    success, rvec, _, _ = cv2.solvePnPRansac(
        model_points, image_points, camera_matrix, dist_coeffs)

    rmat, _ = cv2.Rodrigues(rvec)
    proj_matrix = np.hstack((rmat, np.zeros((3, 1))))
    _, _, _, _, _, _, angles = cv2.decomposeProjectionMatrix(proj_matrix)

    return angles  # pitch, yaw, roll

def get_eye_direction(landmarks, iw):
    left_eye = landmarks[263]  # left eye outer
    right_eye = landmarks[33]  # right eye outer
    eye_mid_x = (left_eye[0] + right_eye[0]) / 2
    eye_mid_x_norm = eye_mid_x / iw  # Normalize to 0-1 range

    if eye_mid_x_norm < 0.45:
        return "Looking Left"
    elif eye_mid_x_norm > 0.55:
        return "Looking Right"
    else:
        return "Looking Center"

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame)

    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        face = frame[y1:y2, x1:x2]

        if face.shape[0] == 0 or face.shape[1] == 0:
            continue

        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        result_mesh = face_mesh.process(face_rgb)

        if result_mesh.multi_face_landmarks:
            for landmarks in result_mesh.multi_face_landmarks:
                ih, iw, _ = face.shape
                coords = [(p.x * iw, p.y * ih) for p in landmarks.landmark]

                try:
                    angles = get_head_pose(face, coords)
                    pitch, yaw, roll = [a[0] for a in angles]
                    if abs(yaw) > 5:
                        cheat = "⚠️ Head turned!"
                        print(f"Yaw: {yaw:.2f} - Cheat: {cheat}")
                    elif abs(pitch) > 7:
                        cheat = "⚠️ Looking up/down!"
                        print(f"Pitch: {pitch:.2f} - Cheat: {cheat}")
                    else:
                        cheat = ""
                except:
                    cheat = "Head Pose Error"
                    print("Head Pose Error")

                eye_dir = get_eye_direction(coords, iw)
                if eye_dir != "Looking Center":
                    cheat = f"⚠️ {eye_dir}"
                    print(f"Eye Direction: {eye_dir} - Cheat: {cheat}")
                else:
                    cheat = ""
                    print("Eye Direction: Looking Center")

                # Draw box and alert
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, eye_dir, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                if cheat:
                    cv2.putText(frame, cheat, (x1, y2 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("YOLO + MediaPipe Cheat Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.

0: 480x640 1 person, 2742.2ms
Speed: 19.9ms preprocess, 2742.2ms inference, 12.4ms postprocess per image at shape (1, 3, 480, 640)
Eye Direction: Looking Center

0: 480x640 1 person, 247.9ms
Speed: 4.9ms preprocess, 247.9ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
Head Pose Error
Eye Direction: Looking Left - Cheat: ⚠️ Looking Left

0: 480x640 1 person, 214.9ms
Speed: 3.3ms preprocess, 214.9ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)
Eye Direction: Looking Left - Cheat: ⚠️ Looking Left

0: 480x640 1 person, 1 tie, 419.3ms
Speed: 5.8ms preprocess, 419.3ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
Eye Direction: Looking Left - Cheat: ⚠️ Looking Left

0: 480x640 1 person, 210.1ms
Speed: 3.9ms preprocess, 210.1ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)
Eye Direction: Looking Left - Cheat: ⚠️ Looking Left

0: 480x640 1 person, 228.9ms
Speed: 3.9ms preprocess, 228.9ms inference

In [13]:
# #just for direction

# import cv2
# import numpy as np
# from ultralytics import YOLO
# import mediapipe as mp

# # Load YOLO model
# model = YOLO("yolov8n.pt")  # Use a face-trained model if you have one

# # Initialize webcam
# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     print("Error: Cannot open webcam.")
#     exit()

# # Initialize MediaPipe Face Mesh
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
#                                   max_num_faces=1,
#                                   refine_landmarks=True,
#                                   min_detection_confidence=0.5,
#                                   min_tracking_confidence=0.5)

# def get_head_pose(image, landmarks):
#     image_points = np.array([
#         landmarks[1],    # Nose tip
#         landmarks[152],  # Chin
#         landmarks[33],   # Right eye outer corner
#         landmarks[263],  # Left eye outer corner
#         landmarks[78],   # Right mouth corner
#         landmarks[308]   # Left mouth corner
#     ], dtype="double")

#     model_points = np.array([
#         (0.0, 0.0, 0.0),             # Nose tip
#         (0.0, -330.0, -65.0),        # Chin
#         (225.0, 170.0, -135.0),      # Right eye outer corner
#         (-225.0, 170.0, -135.0),     # Left eye outer corner
#         (150.0, -150.0, -125.0),     # Right mouth corner
#         (-150.0, -150.0, -125.0)     # Left mouth corner
#     ])

#     focal_length = image.shape[1]
#     center = (image.shape[1] / 2, image.shape[0] / 2)
#     camera_matrix = np.array([
#         [focal_length, 0, center[0]],
#         [0, focal_length, center[1]],
#         [0, 0, 1]
#     ], dtype="double")

#     dist_coeffs = np.zeros((4, 1))
#     success, rvec, _, _ = cv2.solvePnPRansac(
#         model_points, image_points, camera_matrix, dist_coeffs)

#     rmat, _ = cv2.Rodrigues(rvec)
#     proj_matrix = np.hstack((rmat, np.zeros((3, 1))))
#     _, _, _, _, _, _, angles = cv2.decomposeProjectionMatrix(proj_matrix)

#     return angles  # pitch, yaw, roll

# def get_eye_direction(landmarks, iw):
#     left_eye = landmarks[263]  # left eye outer
#     right_eye = landmarks[33]  # right eye outer
#     eye_mid_x = (left_eye[0] + right_eye[0]) / 2
#     eye_mid_x_norm = eye_mid_x / iw  # Normalize to 0-1 range

#     if eye_mid_x_norm < 0.45:
#         return "Looking Left"
#     elif eye_mid_x_norm > 0.55:
#         return "Looking Right"
#     else:
#         return "Looking Center"

# print("Press 'q' to quit.")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = model(frame)

#     for box in results[0].boxes.xyxy:
#         x1, y1, x2, y2 = map(int, box)
#         face = frame[y1:y2, x1:x2]

#         if face.shape[0] == 0 or face.shape[1] == 0:
#             continue

#         face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
#         result_mesh = face_mesh.process(face_rgb)

#         if result_mesh.multi_face_landmarks:
#             for landmarks in result_mesh.multi_face_landmarks:
#                 ih, iw, _ = face.shape
#                 coords = [(p.x * iw, p.y * ih) for p in landmarks.landmark]

#                 try:
#                     angles = get_head_pose(face, coords)
#                     pitch, yaw, roll = [a[0] for a in angles]
#                     if abs(yaw) > 5:
#                         cheat = "⚠️ Head turned!"
#                         print(f"Yaw: {yaw:.2f} - Cheat: {cheat}")
#                     elif abs(pitch) > 7:
#                         cheat = "⚠️ Looking up/down!"
#                         print(f"Pitch: {pitch:.2f} - Cheat: {cheat}")
#                     else:
#                         cheat = ""
#                 except:
#                     cheat = "Head Pose Error"
#                     print("Head Pose Error")

#                 eye_dir = get_eye_direction(coords, iw)
#                 if eye_dir != "Looking Center":
#                     cheat = f"⚠️ {eye_dir}"
#                     print(f"Eye Direction: {eye_dir} - Cheat: {cheat}")
#                 else:
#                     cheat = ""
#                     print("Eye Direction: Looking Center")

#                 # Draw box and alert
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 cv2.putText(frame, eye_dir, (x1, y1 - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
#                 if cheat:
#                     cv2.putText(frame, cheat, (x1, y2 + 20),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

#     cv2.imshow("YOLO + MediaPipe Cheat Detection", frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [14]:
# import cv2
# import numpy as np
# from ultralytics import YOLO
# import mediapipe as mp

# # Load YOLO model
# model = YOLO("yolov8n.pt")  # Use a face-trained model if you have one

# # Initialize webcam
# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     print("Error: Cannot open webcam.")
#     exit()

# # Initialize MediaPipe Face Mesh
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
#                                   max_num_faces=1,
#                                   refine_landmarks=True,
#                                   min_detection_confidence=0.5,
#                                   min_tracking_confidence=0.5)

# def get_head_pose(image, landmarks):
#     image_points = np.array([
#         landmarks[1],    # Nose tip
#         landmarks[152],  # Chin
#         landmarks[150],  # Right eye right corner
#         landmarks[11],   # Left eye left corner
#         landmarks[287],  # Right mouth corner
#         landmarks[57]    # Left mouth corner
#     ], dtype="double")

#     model_points = np.array([
#         (0.0, 0.0, 0.0),             # Nose tip
#         (0.0, -330.0, -65.0),        # Chin
#         (225.0, 170.0, -135.0),      # Right eye
#         (-225.0, 170.0, -135.0),     # Left eye
#         (150.0, -150.0, -125.0),     # Right mouth
#         (-150.0, -150.0, -125.0)     # Left mouth
#     ])

#     focal_length = image.shape[1]
#     center = (image.shape[1] / 2, image.shape[0] / 2)
#     camera_matrix = np.array([
#         [focal_length, 0, center[0]],
#         [0, focal_length, center[1]],
#         [0, 0, 1]
#     ], dtype="double")

#     dist_coeffs = np.zeros((4, 1))
#     success, rvec, _, _ = cv2.solvePnPRansac(
#         model_points, image_points, camera_matrix, dist_coeffs)

#     rmat, _ = cv2.Rodrigues(rvec)
#     proj_matrix = np.hstack((rmat, np.zeros((3, 1))))
#     _, _, _, _, _, _, angles = cv2.decomposeProjectionMatrix(proj_matrix)

#     return angles  # pitch, yaw, roll

# def get_eye_direction(landmarks, iw):
#     left_eye = landmarks[11]
#     right_eye = landmarks[200]
#     eye_mid_x = (left_eye[0] + right_eye[0]) / 2
#     eye_mid_x_norm = eye_mid_x / iw  # Normalize to 0-1 range

#     if eye_mid_x_norm < 0.4:
#         return "Looking Left"
#     elif eye_mid_x_norm > 0.6:
#         return "Looking Right"
#     else:
#         return "Looking Center"


# print("Press 'q' to quit.")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = model(frame)

#     for box in results[0].boxes.xyxy:
#         x1, y1, x2, y2 = map(int, box)
#         face = frame[y1:y2, x1:x2]

#         if face.shape[0] == 0 or face.shape[1] == 0:
#             continue

#         face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
#         result_mesh = face_mesh.process(face_rgb)

#         if result_mesh.multi_face_landmarks:
#             for landmarks in result_mesh.multi_face_landmarks:
#                 ih, iw, _ = face.shape
#                 coords = [(int(p.x * iw), int(p.y * ih)) for p in landmarks.landmark]

#                 # Head pose estimation
#                 try:
#                     angles = get_head_pose(face, coords)
#                     pitch, yaw, roll = [a[0] for a in angles]
#                     if abs(yaw) > 20:
#                         cheat = "⚠️ Head turned!"
#                         print(f"Yaw: {yaw} - Cheat: {cheat} Head truned")
#                     elif abs(pitch) > 20:
#                         cheat = "⚠️ Looking up/down!"
#                         print(f"Pitch: {pitch} - Cheat: {cheat} Looking up/down")
#                     else:
#                         cheat = ""
#                 except:
#                     cheat = "Head Pose Error"
#                     print("Head Pose Error")

#                 # Eye direction
#                 # eye_dir = get_eye_direction(coords)
#                 eye_dir = get_eye_direction(coords, iw)
#                 if eye_dir != "Looking Center":
#                     cheat = f"⚠️ {eye_dir}"
#                     print(f"Eye Direction: {eye_dir} - Cheat: {cheat}")
#                 else:
#                     cheat = ""
#                     print("Eye Direction: Looking Center")

#                 # Draw box and alert
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 cv2.putText(frame, eye_dir, (x1, y1 - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
#                 if cheat:
#                     cv2.putText(frame, cheat, (x1, y2 + 20),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

#     cv2.imshow("YOLO + MediaPipe Cheat Detection", frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
